# Bicing data cleaning
## The objective of this notebook
This notebook goes through the cleaning process of the Bicing usage dataset. 
You can find the source of the dataser here: https://opendata-ajuntament.barcelona.cat/data/en/dataset/estat-estacions-bicing

## What to achieve
The idea is to try to identify different trends on the behaviour of the stations during a regular day. Initially we suppose that stations will behave differently according to their location (is it a residencial station? Or an office one?). In order to do so we need to identify which data will help us answer this.

In [3]:
#Importing the required libraries
import pandas as pd
import seaborn as sns
import datetime as dt


In [4]:
octubre = pd.read_csv("..\\Dataset\\2019_10_Octubre_BicingNou_ESTACIONS.csv",encoding="utf_8")

FileNotFoundError: [Errno 2] File b'..\\Dataset\\2019_10_Octubre_BicingNou_ESTACIONS.csv' does not exist: b'..\\Dataset\\2019_10_Octubre_BicingNou_ESTACIONS.csv'

In [ ]:
#Checking how does the dataset look like
octubre.info()

In [ ]:
octubre.head()

In [ ]:
octubre.isnull().sum()

In [ ]:
#We want the columns "last_reported" and "last_updated" to be datetime. At the moment they are values in Unix Epoch time.
#In order to transform it, we should change the type from float to integer, and then to string (we want to make sure no .0)
octubre.last_reported = octubre.last_reported.astype(int)
octubre.last_reported = octubre.last_reported.astype(str)
octubre.last_updated = octubre.last_updated.astype(str)
octubre.dtypes


In [ ]:
#Transforming from Unix Epoch time to datetime
octubre.last_reported = pd.to_datetime(octubre.last_reported, unit = 's')
octubre.last_updated = pd.to_datetime(octubre.last_updated, unit = 's')

In [ ]:
#We play with a particular station, in this case the one I use from home, with id 381 [Each station has an id that can be
#found on the Bicing app.]
home = octubre.query('station_id== 381')

In [ ]:
#Trying to visualize the number of bikes available per last_reported. It shows the whole month, which is not clear.
home_plot = home[["last_reported","num_bikes_available"]]
home_plot.plot(x='last_reported', y='num_bikes_available', style='-')

In [ ]:
#We will check Ironhack's closest station and see how does it behave on a regular day.
ironhack = octubre[octubre['station_id'] == 211]
ih_bikes = ironhack[["last_reported","num_bikes_available"]]
ih_bikes["last_reported"] = ih_bikes["last_reported"].astype(str)
ih_bikes_day = ih_bikes[ih_bikes["last_reported"].str.startswith('2019-10-18')]

In [ ]:
ih_bikes_day.plot(x='last_reported', y='num_bikes_available', style='-')

There are 4 moments of the day, on which two are busier and two the station is almost empty.

In [ ]:
#By doing the same on the 'Home' station, which is initially a residential area, what do we see?
home = octubre.query('station_id== 381')
home_day = home_day[home_day.last_updated.dt.day == 18]
home_day.plot(x='last_reported', y='num_bikes_available', style='-')

There are three moments, a more full one at the very beginning of the day, a quick drop and emptier time during the day, and slowly gets more bikes towards the end of the day.

# Deleting unneeded columns 
We will initially need:
* Which station are we analyzing "station_id"
* How many bikes at the station "num_bikes_available"
* How many docks available "num_docks_available"
* We will check if "is_installed", "is_renting" and "is_returning" are actually relevant or if we can keep "status"
* We will keep only last_updated to as it seems to be the shared time between stations

- We will not distinguish between mechanical and ebikes at the moment.
- We will drop last_reported and is_charging_station at the moment-

In [ ]:
octubre.head()

In [ ]:
#Dropping initially unneeded columns
octubre.drop(columns=["num_bikes_available_types.mechanical","num_bikes_available_types.ebike","last_reported","is_charging_station"],inplace=True)

In [ ]:
#Let's check values of 'status'
octubre.status.unique()

In [ ]:
#At the moment, our most priority is to know if station is working or not. We will simplify this column using binary values
#0 will mean "is not working" and 1 will equal to 'IN_Service'

In [ ]:
#We make a list of the 'not working' stations, and replace the values for a 0, do the same for 'IN_SERVICE' as 1
not_working = ['PLANNED', 'NOT_IN_SERVICE', 'MAINTENANCE']
octubre.replace(to_replace=not_working,value=0,inplace=True)
octubre.replace(to_replace='IN_SERVICE',value=1,inplace=True)

In [ ]:
octubre.status.value_counts()

In [ ]:
#Now that we have all the columns as numbers, we can check their correlation and decide if we can drop
#is_installed, is_renting and is_returning

In [ ]:
octubre.corr()

We see how 'status' is very correlated with 'is installed', 'is_renting' and 'is_returning' for this reason it makes no sense to keep them all 4 and we can consider 'status' as the one that defines if the station is working or not.

In [ ]:
octubre.drop(columns=["is_installed","is_renting","is_returning"],inplace=True)

In [ ]:
octubre.head()

In [ ]:
#We will keep 'ttl' as number of docks in the station just in case we decide to play with % in the future.

# How do we manage time?

In [ ]:
#There are 8916 total different timeframes at the moment
len(octubre.last_updated.value_counts())

In [ ]:
#We are seeing the dataset contains value from September, we will delete that
octubre.last_updated.head()

In [ ]:
octubre = octubre[octubre.last_updated.dt.month == 10]

In [ ]:
octubre.head()

In [ ]:
octubre.to_csv("..\\Dataset\\October_cleaned.csv",encoding="utf_8")